Playing around with data to learn more about PCA

Author: Emily J King, https://www.math.colostate.edu/~king/

Additional file needed: makepcadata.py

This lab is written to explicitly use SVD in the computation of PCA.  There are packages (e.g., sklearn) which have PCA implementations, but then the mathematics behind PCA are hidden.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
from numpy.linalg import svd
from makepcadata import makepcadata

Generating noisy data around the line y=2x+4 in R^2 and then plotting the data.  

By changing m and sigma, you may change the slope and noise level, respectively, of the data.

In [ ]:
m=2 # slope of the line
b=4 # y-intercept of the line
sigma=0.1 # noise level
X=makepcadata(np.array([[1],[m]]), 100, sigma)+np.array([[0],[b]])

In [ ]:
ax=plt.gca()
ax.grid('off')

plt.plot(X[0,:],X[1,:],'o')
plt.title("Along the line y="+str(m)+"x. Noise level="+str(sigma))

We compute principal component analysis (PCA) of the data by forming the data matrix with data points as columns, centering the data, and computing the singular value decomposition (SVD) of the centered data matrix.

In [ ]:
Xc=X-np.atleast_2d(np.mean(X, axis=1)).T #np.atleast_2d(BLAH).T converts 1D array BLAH into column vector
U, s, VT = svd(Xc, full_matrices=True) 
S = np.zeros(X.shape) #creating Sigma matrix from list of singular values
np.fill_diagonal(S,s)
print("The singular values are "+str(s)) 
print("The singular values are "+str(s/norm(s))) 
print("The (unit norm) principal components are the columns of")
print(U) 
print("Double checking that the SVD is indeed correct by the centered data matrix and USV^T: "+str(np.allclose(Xc,U@S@VT)))

We notice that the first singular value is much larger than the second.  This suggests that the points are modeled well by a one-dimensional affine subspace (i.e., a line).  This makes sense as the data was generated as points from a line with added noise.

To project the data onto PCA affine line of best fit, we project each of the centered points onto the span of the first column of U, then add back in the average values which had been subtracted off when centering.

In [ ]:
Ut=np.transpose(np.array([U[:,0]])) # truncate to first column
St=S[0,0] # truncate to upper left entry
Vt=np.array([VT[0]]) # truncate to first row of transposed matrix

Xt = (Ut*St)@Vt+np.atleast_2d(np.mean(X, axis=1)).T # project down to span of first column of U and add back in the mean of the original vectors

ax=plt.gca()
ax.grid('off')

plt.plot(Xt[0,:],Xt[1,:],'o')
plt.title("Projection to PCA line of best fit of data generated along line y="+str(m)+"x+"+str(b)+".\n Noise level="+str(sigma))


We compare the slope of the line fit by PCA with the slope of the line used to generate the data.

In [ ]:
print("The slope of the line used to generate the data is "+str(m)+",\n while the slope of the fitted line is "+str(U[1,0]/U[0,0]))

Play around with the slope, intercept, and noise level of the generated data in the model above.  In particular, note how changes to the noise level affect the relative sizes of the singular values as well as the slope of the fitted line.

Generating data by randomly drawing points from a fixed two-dimensional linear subspace in R^4 and then adding Gaussian noise.  

We can't visualize points in R^4, so we will only be able to plot projections of our data.

In [ ]:
sigma=0.1 # noise level
W=np.linalg.qr(np.random.randn(4,2))[0] # generate orthonormal basis of random 2D subspace of R^4
X=makepcadata(W, 1000, sigma)

We compute principal component analysis (PCA) of the data by forming the data matrix with data points as columns, centering the data, and computing the singular value decomposition (SVD) of the centered data matrix.

In [ ]:
Xc=X-np.atleast_2d(np.mean(X, axis=1)).T
U, s, VT = svd(Xc, full_matrices=True) 
S = np.zeros(X.shape)
np.fill_diagonal(S,s)
print("The singular values are "+str(s)) 
print("The singular values are "+str(s/norm(s))) 
print("The (unit norm) principal components are the columns of")
print(U) 

We notice that the first two singular values are much larger than the last two.  This suggests that the points are modeled well by a two-dimensional affine subspace (i.e., a plane).

To project the data onto PCA affine plane of best fit, we project each of the centered points onto the span of the first two columns of U, then add back in the average values which had been subtracted off when centering.   If we just project down to the span of the first two columns of U, the resulting numbers are called the scores.  

However, we cannot visualize this two-dimensional subspace of R^4.  Thus, we visualize a projection of the data onto R^2 by using the inner products of the centered data with the first two columns of U as the coordinates in R^2.  The points are randomly spread in this subspace due to the way that makepcadata generates the random data sets, as seen in the plot.

In [ ]:
Ut=U[:,0:2] # truncate to first two columns
St=S[0:2,0:2] # truncate to upper left 2x2 entries
Vt=VT[0:2] # truncate to first two rows of transposed matrix

Xt = St@Vt

ax=plt.gca()
ax.grid('off')

plt.plot(Xt[0,:],Xt[1,:],'o')
plt.title("Projection to coords of first two columns of U. Noise level="+str(sigma))

We also compute the covariance matrices of the projections of the centered points onto the subspace spanned by the first two columns of U and onto the last two columns. Note that the variances are much  larger for the projection onto the first two columns of U; this corresponds to the intuition that those directions are responsible for most of the variance of the points.  Since the columns of U are orthogonal, the covariances are 0 in all cases.

In [ ]:
print("The covariance matrix of the projection of the points\n onto the first two columns of U:")
print(np.cov(Xt))

In [ ]:
print("The covariance matrix of the projection of the points\n onto the last two columns of U:")
print(np.cov(S[2:4,2:4]@VT[2:4]))

Finally, we compute the Frobenius distance between the orthgonal projection onto the two-dimensional subspace of R^4 used to generate the points and orthogonal projection onto the two-dimensional subspace found by PCA.

In [ ]:
print("The Frobenius distance between the projection onto the two-dimensonal subspace\n used to generate the data and the subspace fit by PCA is "+str(norm(Ut@Ut.T-W@W.T)))

Play around with the noise level of the generated data in the model above.  In particular, note how changes to the noise level affect the relative sizes of the singular values as well as the Frobenius distance between the projections onto the original and learned subspaces.

Generating data by first generating a random four-dimensional subspace of R^50 and then generating data by randomly drawing points from this subspace and adding Gaussian noise.  

We can't visualize points four-dimensional spaces, let alone R^50, so we won't plot the points.

In [ ]:
sigma=0.1
W=np.linalg.qr(np.random.randn(50,4))[0] 
X=makepcadata(W, 1000, sigma)

We compute principal component analysis (PCA) of the data by forming the data matrix with data points as columns, centering the data, and computing the singular value decomposition (SVD) of the centered data matrix.

In [ ]:
Xc=X-np.atleast_2d(np.mean(X, axis=1)).T
U, s, VT = svd(Xc, full_matrices=True) 
S = np.zeros(X.shape)
np.fill_diagonal(S,s)

Since there are 50 singular values, it is more helpful to look at a plot of them rather than a list.  Such a plot of singular values is called a scree plot.

In [ ]:
ax=plt.gca()
ax.grid('off')

plt.plot(s,'o')
plt.title("Scree plot of singular values of points near random 4D subspace of R^50.\n Noise level="+str(sigma))

We notice that the first four singular values are much larger than the remaining 46.  This suggests that the points are modeled well by a four-dimensional subspace.

We can't really plot points or their projections in a meaningful way, but we can compute the 50 variances of the data projected onto the 50 difference columns of U.

In [ ]:
ax=plt.gca()
ax.grid('off')

plt.plot(np.var(S@VT,axis=1),'o')
plt.title("Variances of data projected onto each column of U.\n Noise level="+str(sigma))

Note the similarity of the shape of the scree plot and the plot of the variances.  This is because they are both giving the same message: that the subspace spanned by the first four columns of U describes most of the variance of the points.

Finally, we compute the Frobenius distance between the orthgonal projection onto the random four-dimensional subspace of R^50 used to generate the points and orthogonal projection onto the four-dimensional subspace found by PCA.

In [ ]:
Ut=U[:,0:4] # truncate to first four columns

print("The Frobenius distance between the projection onto the four-dimensonal subspace\n used to generate the data and the subspace fit by PCA is "+str(norm(Ut@Ut.T-W@W.T)))


Play around with the noise level of the generated data in the model above.  In particular, note how changes to the noise level affect the relative sizes of the singular values as well as the Frobenius distance between the projections onto the original and learned subspaces.